## **Thư viện**

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
from bs4 import BeautifulSoup
import json
from urllib.parse import urlparse
import csv

**Lấy source page trang `https://mobilecity.vn/dien-thoai`**

In [4]:
# Tạo instance của trình duyệt
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Truy cập vào trang web
url = "https://mobilecity.vn/dien-thoai"
driver.get(url)

# Hàm nhấn nút "Xem thêm" nếu có thể
def click_see_more_button():
    try:
        # Tìm nút "Xem thêm điện thoại"
        see_more_button = driver.find_element(By.ID, "product_view_more")
        if see_more_button.is_displayed():
            # Nhấn nút
            see_more_button.click()
            # Đợi một chút để trang tải thêm sản phẩm
            time.sleep(2)
            return True
    except (NoSuchElementException, ElementClickInterceptedException):
        # Không tìm thấy nút hoặc không thể nhấn
        return False
    return False

# Liên tục nhấn nút "Xem thêm" cho đến khi không còn nút nào
while click_see_more_button():
    pass

# Lấy source page sau khi đã nhấn hết các nút "Xem thêm"
page_source = driver.page_source
# Đóng trình duyệt
driver.quit()

**Thu thập đường dẫn đến từng trang điện thoại chi tiết**

In [5]:
# Phân tích cú pháp HTML
soup = BeautifulSoup(page_source, "html.parser")

# Tìm tất cả các sản phẩm
product_items = soup.find_all("div", class_="product-item-info")

# Lưu các liên kết vào danh sách
links = []

for item in product_items:
    # Tìm thẻ <a> chứa liên kết đến chi tiết sản phẩm
    link_tag = item.find("a", href=True)
    if link_tag:
        # Lấy giá trị href
        link = link_tag['href']
        links.append(link)

In [6]:
def is_valid_url(url):
    """Check if the URL is valid."""
    parsed = urlparse(url)
    return bool(parsed.scheme and parsed.netloc)

**Thu thập các thuộc tính của từng sản phẩm**

In [7]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

product_data = []
counter = 0  

for link in links:
    link = link.strip()  # Xóa khoảng trắng đầu và cuối
    if not is_valid_url(link):
        print(f"Invalid URL: {link}")
        continue  # Skip invalid URLs

    driver.get(link)
    time.sleep(3)  # Chờ trang tải

    # Lấy HTML trang hiện tại
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Thu thập tên sản phẩm
    try:
        product_title = soup.find("h1", class_="title").text.strip()
    except AttributeError:
        product_title = "Không có tên sản phẩm"

    # Thu thập giá sản phẩm (giá mới và giá cũ)
    try:
        price_new = soup.find("p", class_="price").text.strip()
    except AttributeError:
        price_new = "Không có giá mới"

    try:
        price_old = soup.find("p", class_="price-old").text.strip()
    except AttributeError:
        price_old = "Không có giá cũ"

    # Thu thập màu sắc
    try:
        color_items = soup.find_all("div", class_="color-item")
        colors = [color['data-title'] for color in color_items]
    except AttributeError:
        colors = ["Không có màu sắc"]

    # Lấy thông tin về các phiên bản bộ nhớ
    try:
        storage_items = soup.find_all('div', class_='storage-item attribute-item')
        storage_options = [item.text.strip() for item in storage_items]
    except AttributeError:
        storage_options = ["Không có phiên bản bộ nhớ"]

    # Lấy thông tin về thời gian bảo hành
    try:
        warranty_text = soup.find('span', class_='warranty-content-default').get_text(strip=True)
    except AttributeError:
        warranty_text = "Không có thông tin bảo hành"

    # Lấy thông số kỹ thuật
    try:
        specs_table = soup.find('div', class_='product-info-content').find('table')
        specs = {}
        if specs_table:
            rows = specs_table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:
                    specs[cols[0].text.strip()] = cols[1].text.strip()
    except AttributeError:
        specs = {"Không có thông số kỹ thuật": "N/A"}

    # Thu thập thông tin đánh giá
    try:
        rating = soup.find('div', class_='comment-vote__star-number').get_text(strip=True)
    except AttributeError:
        rating = "Không có đánh giá"

    # Thu thập thông tin số lượt đánh giá và hỏi đáp
    try:
        total_reviews = soup.find('div', class_='comment-vote__star-total').get_text(strip=True)
        total_reviews = total_reviews.replace("đánh giá và hỏi đáp", "").strip()
    except AttributeError:
        total_reviews = "Không có thông tin đánh giá và hỏi đáp"

    product_data.append({
        "Tên sản phẩm": product_title,
        "Giá mới": price_new,
        "Giá cũ": price_old,
        "Màu sắc": ", ".join(colors),
        "Các phiên bản bộ nhớ": ", ".join(storage_options),
        "Thời gian bảo hành": warranty_text,
        "Thông số kỹ thuật": json.dumps(specs, ensure_ascii=False),
        "Đánh giá": rating,
        "Số lượt đánh giá và hỏi đáp": total_reviews,
        "Đường dẫn":link,
    })
    counter += 1
    if counter % 100 == 0:
        print(f"Đã thu thập thành công {counter} sản phẩm")

# In thông báo khi hoàn thành
print("Hoàn thành!!!")

Đã thu thập thành công 100 sản phẩm
Đã thu thập thành công 200 sản phẩm
Đã thu thập thành công 300 sản phẩm
Đã thu thập thành công 400 sản phẩm
Đã thu thập thành công 500 sản phẩm
Đã thu thập thành công 600 sản phẩm
Đã thu thập thành công 700 sản phẩm
Đã thu thập thành công 800 sản phẩm
Đã thu thập thành công 900 sản phẩm
Đã thu thập thành công 1000 sản phẩm
Đã thu thập thành công 1100 sản phẩm


KeyboardInterrupt: 

**Lưu dữ liệu thu thập được vào file `phone_data.csv`**

In [ ]:
# Lưu dữ liệu vào file CSV
with open('phone_data.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = product_data[0].keys()  # Lấy các tên trường từ phần tử đầu tiên
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in product_data:
        writer.writerow(data)

# Đóng trình duyệt
driver.quit()